In [1]:
import torch

In [2]:
from utils.plotter import MetricsPlotter
from utils.logger import Logger
from utils.utils import set_settings
from data import experiment
from utils.config import get_config

args = get_config()
set_settings(args)
# logger plotter
exper_detail = f"Dataset : {args.dataset.upper()}, Model : {args.model}, Train_size : {args.train_size}, Bs : {args.bs}, Rank : {args.rank}, "
exper_detail += f"Train Device : {args.train_device} "
log_filename = f'Model_{args.model}_T{args.train_size}_R{args.rank}'
log = Logger(log_filename, exper_detail, args)
plotter = MetricsPlotter(log_filename, args)
args.log = log
log(str(args.__dict__))

|2024-08-22 16:08:28| {
     'att': 0, 'bs': 32, 'classification': False, 'cross': 0, 'dataset': cpu,
     'debug': False, 'decay': 0.0001, 'density': 0.8, 'device': cpu, 'device_name': core-i7-7820x,
     'epochs': 1000, 'eval_device': desktop-cpu-core-i9-13900k-fp32, 'experiment': False, 'graph_encoder': gat, 'heads': 0,
     'inductive': False, 'llm': 1, 'logger': None, 'loss_func': L1Loss, 'lr': 0.001,
     'model': ours, 'num_preds': 1, 'num_windows': 12, 'op_encoder': embed, 'optim': AdamW,
     'order': 4, 'path': ./datasets/, 'patience': 100, 'program_test': False, 'rank': 100,
     'record': True, 'rounds': 5, 'seed': 0, 'train_device': desktop-cpu-core-i7-7820x-fp32, 'train_size': 500,
     'verbose': 0, 'visualize': True,
}
|2024-08-22 16:08:28| {'classification': False, 'visualize': True, 'inductive': False, 'bs': 32, 'lr': 0.001, 'decay': 0.0001, 'loss_func': 'L1Loss', 'optim': 'AdamW', 'path': './datasets/', 'dataset': 'cpu', 'train_size': 500, 'density': 0.8, 'logger': '

In [3]:
from train_model import Model
from data import DataModule

# Initialize
exper = experiment(args)
datamodule = DataModule(exper, args)
model = Model(datamodule, args)

desktop-cpu-core-i7-7820x-fp32.pickle
|2024-08-22 16:08:28| Train_length : 500 Valid_length : 1528 Test_length : 13256 Max_value : 0.01
加载存储数据 tensor([[0.6143, 0.2500, 0.5000, 0.2750, 0.3750]])


/Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [4]:
model.load_state_dict(torch.load('./checkpoints/ours/Model_ours_gpu_S200_R100_O3_round_4.pt', map_location=torch.device('cpu')))

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/ours/Model_ours_T50_R300_round_0.pt'

In [ ]:
idx = 32

In [ ]:
sample_input = datamodule.train_loader.dataset[idx]
graph, features, values = sample_input
print(graph, features)
print(model(graph, features.reshape(1, -1)))

In [ ]:
model.model.aug_data

In [ ]:
def get_embeds_importance(model, sample_input):
    import torch
    
    graph, features, values = sample_input
    features = features.reshape(1, -1).long()
    
    # 清除模型中所有参数的梯度
    model.zero_grad()
    
    # 确保 info_embeds 和 graph_embeds 的梯度可以计算
    batch_size = features.shape[0]
    aug_data_repeated = model.model.aug_data.expand(batch_size, -1).to(model.model.args.device)
    info_embeds = model.model.info_encoder(aug_data_repeated)
    graph_embeds = model.model.graph_encoder(graph, features)
    
    # 确保在前向传播之前设置 requires_grad=True
    info_embeds.requires_grad_(True)
    graph_embeds.requires_grad_(True)
    
    # 在非叶子张量上保留梯度
    info_embeds.retain_grad()
    graph_embeds.retain_grad()
    
    # 组合嵌入，进行前向传播
    embeds = torch.cat([graph_embeds, info_embeds], dim=1)
    output = model.model.predictor(embeds)
    
    # 使用人为定义的损失函数，比如对输出的简单加权平均
    loss = output.mean()  # 简单的均值作为损失
    
    # 反向传播计算损失的梯度
    loss.backward()
    
    # 获取两个嵌入的梯度并计算它们的重要性
    info_importance = info_embeds.grad.abs().sum().item()
    graph_importance = graph_embeds.grad.abs().sum().item()
    
    total_importance = info_importance + graph_importance
    graph_importance /= total_importance
    info_importance /= total_importance
    
    print("LLMs Embeds Importance:", info_importance)
    print("Graph Embeds Importance:", graph_importance)
    return graph_importance, info_importance
_, _ = get_embeds_importance(model, sample_input=datamodule.train_loader.dataset[idx])

In [ ]:
def get_node_importance(model, sample_input):
    graph, features, values = sample_input
    features = features.reshape(1, -1).long()
    g, feats = graph, model.model.graph_encoder.dnn_embedding(features).reshape(features.shape[0] * 9, -1)
    attention_weights = []
    for i, (layer, norm, act) in enumerate(zip(model.model.graph_encoder.layers, model.model.graph_encoder.norms, model.model.graph_encoder.acts)):
        # 获取 GAT 的注意力权重
        feats, attn = layer(g, feats, get_attention=True)
        feats = feats.mean(dim=1)  # 聚合多个头的输出
        feats = norm(feats)
        feats = act(feats)
        attention_weights.append(attn)
    batch_sizes = torch.as_tensor(g.batch_num_nodes()).to(model.model.args.device)  # 每个图的节点数
    first_nodes_idx = torch.cumsum(torch.cat((torch.tensor([0]).to(model.model.args.device), batch_sizes[:-1])), dim=0)  # 使用torch.cat来连接Tensor
    first_node_features = feats[first_nodes_idx]
    return first_node_features, attention_weights
first_node_features, attention_weights = get_node_importance(model, sample_input = datamodule.train_loader.dataset[idx])

In [ ]:
graph

In [ ]:
len(attention_weights)

In [ ]:
global_node_attention = attention_weights[-1] 
global_node_attention.shape

In [ ]:
# def aggregate_attention(global_node_attention):
#     # 假设 global_node_attention 的形状是 [num_edges, num_heads, 1]
#     # 对注意力权重在所有头上取平均
#     mean_attention = global_node_attention.mean(dim=1)  # [num_edges, 1]
# 
#     # 初始化节点的重要性
#     node_importance = mean_attention.squeeze(-1)  # 移除最后的单一维度，得到 [num_edges]
# 
#     return node_importance
# 
# # 使用聚合函数计算节点的重要性
# node_importance = aggregate_attention(global_node_attention)
# print("Node importance to the global node:", node_importance.detach().cpu().numpy())

In [ ]:

# def aggregate_node_attention(graph, global_node_attention):
#     # 获取图中的节点数和边数
#     num_nodes = graph.number_of_nodes()
#     num_edges = graph.number_of_edges()
# 
#     # 假设 global_node_attention 的形状是 [num_edges, num_heads, 1]
#     mean_attention = global_node_attention.mean(dim=1).squeeze(-1)  # [num_edges]
#     
#     print(mean_attention)
#     
#     # 初始化每个节点的注意力权重为零
#     node_importance = torch.zeros(num_nodes, device=mean_attention.device)
#     print(node_importance)
#     # 遍历每条边，将注意力权重聚合到对应的节点上
#     for edge_idx in range(num_edges):
#         src, dst = graph.find_edges(edge_idx)  # 获取边的起始节点和目标节点
#         # 累加目标节点（全局节点）的注意力权重
#         node_importance[dst] += mean_attention[edge_idx]
#         print(dst, node_importance[dst])
#     # print(node_importance)
#     
#     return node_importance
# 
# # 使用函数计算每个节点对全局节点的重要性
# node_importance = aggregate_node_attention(graph, global_node_attention)
# print("Node importance to the global node:", node_importance.detach().cpu().numpy())

In [ ]:
def get_device_feature_importance(model, sample_input):
    import torch
    
    # 清除模型中所有参数的梯度
    model.zero_grad()
    
    graph, features, values = sample_input
    
    # 获取 info_importance
    _, info_importance = get_embeds_importance(model, sample_input)
    
    # 设置 aug_data 为 requires_grad=True
    model.model.aug_data.requires_grad = True
    
    # 前向传播计算输出 y
    features = features.reshape(1, -1).long()
    output = model(graph, features)
    
    # 反向传播计算每个输入特征的梯度
    output.backward()
    
    # 获取 aug_data 的梯度并计算重要性
    aug_data_importance = model.model.aug_data.grad.abs()
    
    # 标准化重要性，使其和为1
    aug_data_importance = aug_data_importance / aug_data_importance.sum()
    
    # 打印 aug_data 中每个元素的重要性
    print("Importance of each element in aug_data:", aug_data_importance.detach().numpy())
    
    # 计算每个 aug_data 元素在整个模型中的重要性
    each_device_info_importance = info_importance * aug_data_importance
    each_device_info_importance = each_device_info_importance.squeeze(0).detach().numpy()
    print("Overall importance of each element in aug_data to the model:")
    for i in range(len(each_device_info_importance)):
        print(f"LLMs features-{i} : {each_device_info_importance[i]:.4f}", )
    
get_device_feature_importance(model = model, sample_input = datamodule.train_loader.dataset[idx])

In [ ]:
def get_node_importance(model, sample_input):
    graph, features, values = sample_input
    features = features.reshape(1, -1).long()
    g, feats = graph, model.model.graph_encoder.dnn_embedding(features).reshape(features.shape[0] * 9, -1)
    attention_weights = []
    for i, (layer, norm, act) in enumerate(zip(model.model.graph_encoder.layers, model.model.graph_encoder.norms, model.model.graph_encoder.acts)):
        # 获取 GAT 的注意力权重
        feats, attn = layer(g, feats, get_attention=True)
        feats = feats.mean(dim=1)  # 聚合多个头的输出
        feats = norm(feats)
        feats = act(feats)
        attention_weights.append(attn)
    batch_sizes = torch.as_tensor(g.batch_num_nodes()).to(model.model.args.device)  # 每个图的节点数
    first_nodes_idx = torch.cumsum(torch.cat((torch.tensor([0]).to(model.model.args.device), batch_sizes[:-1])), dim=0)  # 使用torch.cat来连接Tensor
    first_node_features = feats[first_nodes_idx]
    return first_node_features, attention_weights
first_node_features, attention_weights = get_node_importance(model, sample_input = datamodule.train_loader.dataset[idx])


def aggregate_node_attention(model, sample_input, graph, global_node_attention):
    # 获取图中的节点数和边数
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()

    # 假设 global_node_attention 的形状是 [num_edges, num_heads, 1]
    mean_attention = global_node_attention.mean(dim=1).squeeze(-1)  # [num_edges]
    
    print("Mean attention per edge:", mean_attention)
    
    # 初始化每个节点的注意力权重为零
    node_importance = torch.zeros(num_nodes, device=mean_attention.device)
    
    # 遍历每条边，将注意力权重聚合到对应的节点上
    for edge_idx in range(num_edges):
        src, dst = graph.find_edges(edge_idx)  # 获取边的起始节点和目标节点
        if dst == 0:
            # 累加目标节点的注意力权重
            node_importance[src] += mean_attention[edge_idx]
            print(f"Edge {edge_idx:2d}: src={src}, dst={dst}, Attention={mean_attention[edge_idx]:.4f}, accumulated importance={node_importance[src]}")
    
    # 最后可以对 node_importance 进行归一化处理，使其值在合理范围内
    node_importance = node_importance / node_importance.sum()
    
    print("Node importance to the global node:\n", node_importance.detach().cpu().numpy())

    # 获取 graph_importance
    graph_importance, _ = get_embeds_importance(model, sample_input)
    each_device_node_importance = graph_importance * node_importance
    each_device_node_importance = each_device_node_importance.detach().numpy()
    _, key, values = sample_input
    print("Overall importance of each element in graph to the Global Node:")
    for i in range(len(key)):
        print(f"Key={key[i].item()}: {each_device_node_importance[i]:.4f}")
    return node_importance

# 使用函数计算每个节点对全局节点的重要性
node_importance = aggregate_node_attention(model, sample_input, graph, global_node_attention)

In [ ]:
"""
    0 con1 1 con3 2 max3 3 input 4 output 5 None
"""
op_name = ['con1', 'con3', 'max3', 'input', 'output', 'None']

In [2]:
"""
    CPU Clock Frequency, Maximum Turbo Boost Frequency, Number of Cores, Number of Threads, Level 3 Cache, Maximum Memory Bandwidth
"""
device_fetures = ['CPU Clock Frequency', 'Maximum Turbo Boost Frequency', 'Number of Cores', 'Number of Threads', 'Level 3 Cache', ', Maximum Memory Bandwidth']